In [7]:
!pip install pandas numpy yfinance scikit-learn

import signal
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.ensemble import IsolationForest
import time

# --- Configuration ---
# List of stock tickers to monitor
STOCK_TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
# Time interval for fetching data (in seconds)
FETCH_INTERVAL_SECONDS = 60
# Lookback period for anomaly detection (number of data points)
LOOKBACK_PERIOD = 100
# Anomaly detection model parameters (Isolation Forest)
CONTAMINATION_FACTOR = 'auto'

# --- Data Fetching ---
def fetch_stock_data(ticker):
    """Fetches the latest stock data for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        # Get recent historical data, adjust period and interval as needed
        data = stock.history(period='1d', interval='1m')
        if data.empty:
            print(f"Warning: No data fetched for {ticker}")
            return pd.DataFrame()
        # Use the closing price for simplicity
        return data[['Close']].reset_index()
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

# --- Data Processing and Anomaly Detection ---
def process_and_detect(data, ticker, model):
    """Processes data and applies anomaly detection."""
    if data.empty or len(data) < LOOKBACK_PERIOD:
        print(f"Not enough data for {ticker} to perform anomaly detection.")
        return

    # Use the latest `LOOKBACK_PERIOD` data points
    recent_data = data.tail(LOOKBACK_PERIOD).copy()

    # Prepare data for the model
    X = recent_data[['Close']].values

    # Fit the model on the recent data
    model.fit(X)

    # Predict anomalies
    # -1 indicates an outlier, 1 indicates an inlier
    recent_data['anomaly'] = model.predict(X)

    # Check for recent anomalies
    latest_point = recent_data.iloc[-1]
    if latest_point['anomaly'] == -1:
        alert(ticker, latest_point['Close'], latest_point.name)

# --- Alerting System ---
def alert(ticker, price, timestamp):
    """Generates an alert for a detected anomaly."""
    print(f"ANOMALY DETECTED for {ticker}!")
    print(f"  Timestamp: {timestamp}")
    print(f"  Closing Price: {price:.2f}")
    print("-" * 20)

# Set a flag to indicate whether the program should terminate
terminate_flag = False

def signal_handler(signum, frame):
    """Handler to catch termination signals."""
    global terminate_flag
    print(f"Received signal {signum}. Initiating shutdown...")
    terminate_flag = True

# Register signal handlers for graceful shutdown
signal.signal(signal.SIGINT, signal_handler)
signal.signal(signal.SIGTERM, signal_handler)

def start_monitoring():
    """Starts the real-time stock price monitoring loop."""
    global terminate_flag
    print("Starting stock monitoring...")

    # Assuming STOCK_TICKERS and anomaly_models are defined/initialized
    anomaly_models = {}
    for ticker in STOCK_TICKERS:
         anomaly_models[ticker] = IsolationForest(contamination=CONTAMINATION_FACTOR, random_state=42)

    historical_data = {ticker: pd.DataFrame() for ticker in STOCK_TICKERS}

    # Add the termination check to the while loop condition
    while not terminate_flag:
        print(f"\nFetching data at {pd.Timestamp.now()}...")
        for ticker in STOCK_TICKERS:
            # Check terminate_flag inside the inner loop as well
            if terminate_flag:
                break
            latest_data = fetch_stock_data(ticker)

            if not latest_data.empty:
                historical_data[ticker] = pd.concat([historical_data[ticker], latest_data]).drop_duplicates(subset=['Datetime']).sort_values('Datetime').reset_index(drop=True)

                if len(historical_data[ticker]) > LOOKBACK_PERIOD * 2:
                     historical_data[ticker] = historical_data[ticker].tail(LOOKBACK_PERIOD * 2).reset_index(drop=True)

                if ticker in anomaly_models:
                    process_and_detect(historical_data[ticker], ticker, anomaly_models[ticker])
                else:
                    print(f"Model not initialized for {ticker}")

        if not terminate_flag: # Only sleep if not terminating
            print(f"Waiting for {FETCH_INTERVAL_SECONDS} seconds...")
            time.sleep(FETCH_INTERVAL_SECONDS)

    print("Monitoring loop terminated.")

if __name__ == "__main__":
    start_monitoring()

Starting stock monitoring...

Fetching data at 2025-07-08 06:29:06.528500...
ANOMALY DETECTED for AAPL!
  Timestamp: 199
  Closing Price: 209.94
--------------------
ANOMALY DETECTED for MSFT!
  Timestamp: 199
  Closing Price: 497.83
--------------------
ANOMALY DETECTED for GOOGL!
  Timestamp: 199
  Closing Price: 176.79
--------------------
ANOMALY DETECTED for AMZN!
  Timestamp: 199
  Closing Price: 223.48
--------------------
Waiting for 60 seconds...

Fetching data at 2025-07-08 06:30:07.621416...
ANOMALY DETECTED for AAPL!
  Timestamp: 199
  Closing Price: 209.94
--------------------
ANOMALY DETECTED for MSFT!
  Timestamp: 199
  Closing Price: 497.83
--------------------
ANOMALY DETECTED for GOOGL!
  Timestamp: 199
  Closing Price: 176.79
--------------------
ANOMALY DETECTED for AMZN!
  Timestamp: 199
  Closing Price: 223.48
--------------------
Waiting for 60 seconds...
Received signal 2. Initiating shutdown...
Monitoring loop terminated.
